### INCREMENTAL DATA INGESTION

In [0]:
# Retrieve the value entered in the "src" widget
src_value = dbutils.widgets.get("src")

In [0]:
# Read streaming csv data from the specified raw bookings path using Auto Loader,
# with schema stored and evolved at the given checkpoints location
df = spark.readStream.format("cloudFiles") \
  .option("cloudFiles.format", "csv") \
  .option("cloudFiles.schemaLocation", f"/Volumes/workspace/bronze/bronzevolume/{src_value}/checkpoints")\
  .option("cloudFiles.schemaEvolutionMode", "rescue")\
  .load(f"/Volumes/workspace/raw/rawvolume/rawdata/{src_value}/")

In [0]:
# Write the streaming DataFrame to a Delta table in append mode,
# using a trigger to process available data once, with checkpointing for fault tolerance,
# and saving the data to the specified path
df.writeStream.format("delta")\
    .outputMode("append")\
    .trigger(once=True)\
    .option("checkpointLocation", f"/Volumes/workspace/bronze/bronzevolume/{src_value}/checkpoints")\
    .option("path", f"/Volumes/workspace/bronze/bronzevolume/{src_value}/data")\
    .start()